In [1]:
# import plotly.graph_objs as go
# from plotly.offline import plot

# fig = go.Figure([go.Bar(y=[2, 3, 1])],)

# fig.update_layout(
#   autosize=True,   # let the container size drive the plot
#   margin=dict(l=0, r=0, t=0, b=0)
# )
# div_snippet = plot(
#     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# )

# # write div snippet to file
# with open('plotly_div.html', 'w') as f:
#     f.write(div_snippet)

In [2]:
import os
import sys
from railway import *

In [3]:
# Root folder directory
ROOT_DIR = "/home/marco/railway-scheduling"

# Set the current directory to root directory
os.chdir(ROOT_DIR)
sys.path.append(os.getcwd())
print(f"Current working directory: 📂 {os.getcwd()}")

Current working directory: 📂 /home/marco/railway-scheduling


In [4]:
# # Define problem parameters
# N = 10
# T = 10
# J = 10
# P = 2000
# K = 3

# # Name of the file to load
# FILENAME = f"datasets/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"

# # Instantiate the Railway object
# rail = Railway.load(FILENAME)
# # rail.model.setParam('TimeLimit', 60)

# # Print the model in summary
# print(rail)

In [5]:
# Generate a problem

# Constant values
n = 5
periods = 10
jobs = 10
passengers = 2000
K = 3

# Create blueprint model
rail = Railway(n, periods, jobs, passengers, K)

# Generate problem parameters
job_min_time, job_max_time = 1, 2
job_min_length, job_max_length = 1, 2
pause_min_time, pause_max_time = 0, 0
min_demand, max_demand = 0.0, 1.0
min_share, max_share = 1.0, 1.0
min_capacity, max_capacity = 0.5, 0.7
n_max_events = 0
event_min_length, event_max_length = 0, 0

rail.generate(
    job_min_time,
    job_max_time,
    job_min_length,
    job_max_length,
    pause_min_time,
    pause_max_time,
    min_demand,
    max_demand,
    min_share,
    max_share,
    min_capacity,
    max_capacity,
    n_max_events,
    event_min_length,
    event_max_length,
)


# # Load model

# # Define problem parameters
# N = 10
# T = 10
# J = 40
# P = 2000
# K = 3

# # Name of the file to load
# FILENAME = f"../datasets/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"

# # Instantiate the Railway object
# model = Railway.load(FILENAME)

Set parameter Username
Set parameter LicenseID to value 2629256
Academic license - for non-commercial use only - expires 2026-02-27
Problem generated successfully. Remember to set constraints and objective (again).


In [7]:
# Copy before optimizing
# rail2 = Railway.copy(rail)

# Set constraints
rail.set_constraints()

# Set objective
rail.set_objective()

# Silence output
rail.model.setParam('OutputFlag', 0)
rail.model.setParam('Timelimit', 20)

# Most basic model
rail.model.setParam('LPWarmStart',0)
rail.model.setParam('PoolSolutions', 1)
# rail.model.Params.presolve = 0
rail.model.Params.MultiObjPre = 0
rail.model.Params.cuts = 0
rail.model.Params.cutpasses = 0
rail.model.Params.threads = 1
rail.model.Params.heuristics = 0
rail.model.Params.symmetry = 0

# Solve the scheduling problem
performance = rail.optimize()

In [15]:
rail.coords

[(1.0, 0.0),
 (0.8090169943749475, 0.5877852522924731),
 (0.30901699437494745, 0.9510565162951535),
 (-0.30901699437494734, 0.9510565162951536),
 (-0.8090169943749473, 0.5877852522924732),
 (-1.0, 1.2246467991473532e-16),
 (-0.8090169943749475, -0.587785252292473),
 (-0.30901699437494756, -0.9510565162951535),
 (0.30901699437494723, -0.9510565162951536),
 (0.8090169943749473, -0.5877852522924734)]

In [ ]:
import plotly.graph_objects as go
import numpy as np


# Variables
N = rail.N
coords = rail.coords
coords_plot = []
for (x, y), i in zip(coords, N):
	coords_plot.append((x, y, i))
A = rail.A
Ja = rail.Ja
omega_e = rail.omega_e
omega_j = rail.omega_j
J = rail.J
Aj = rail.Aj




# Create figure
fig = go.Figure()

# Create background (gray rectangle + white circle)
fig.add_shape(
    type="rect",
    x0=-1.1, y0=-1.1, x1=1.1, y1=1.1,
    fillcolor="lightgray",
    line=dict(color="lightgray", width=0),
    layer="below"
)

fig.add_shape(
    type="circle",
    x0=-1, y0=-1, x1=1, y1=1,
    fillcolor="white",
    line=dict(color="white", width=0),
    layer="below"
)

# Add dashed unit circle
theta = np.linspace(0, 2 * np.pi, 100)
circle_x = np.cos(theta)
circle_y = np.sin(theta)

fig.add_trace(go.Scatter(
    x=circle_x, y=circle_y,
    mode="lines",
    line=dict(color="black", dash="dash", width=1),
    hoverinfo="skip",
    showlegend=False
))

# Add arcs (gray lines between stations)
for i, j in A:
    i_idx = i - 1  # Adjust for 0-based indexing
    j_idx = j - 1
    x0, y0 = coords[i_idx]
    x1, y1 = coords[j_idx]
    
    # Create hover text
    if Ja is not None:
        jobs_text = f"{Ja[(i, j)]}" if (i, j) in Ja and Ja[(i, j)] else "None"
    else:
        jobs_text = ""
        
    hover_text = f"Arc: ({i}, {j})<br>omega_e: {omega_e[(i, j)]:.2f}<br>omega_j: {omega_j[(i, j)]:.2f}"
    if jobs_text:
        hover_text += f"<br>Jobs: {jobs_text}"
        
    fig.add_trace(go.Scatter(
        x=[x0, x1], y=[y0, y1],
        mode="lines",
        line=dict(color="gray", width=2),
        hovertext=hover_text,
        hovertemplate=hover_text + "<extra></extra>",
        showlegend=False
    ))

# Add colored arcs for jobs (if provided)
if J is not None and Aj is not None:
    # Import color palette similar to tab10
    import plotly.express as px
    colors = px.colors.qualitative.Set1
    
    for j, color in zip(J, colors):
        if j > len(colors):  # Handle case with more jobs than colors
            color = colors[j % len(colors)]
            
        if j in Aj:
            for a in Aj[j]:
                i, k = a
                i_idx = i - 1
                k_idx = k - 1
                x0, y0 = coords[i_idx]
                x1, y1 = coords[k_idx]
                
                fig.add_trace(go.Scatter(
                    x=[x0, x1], y=[y0, y1],
                    mode="lines",
                    line=dict(color=color, width=2),
                    name=f"Job {j}",
                    hoverinfo="text",
                    hovertext=f"Job {j}<br>Arc: ({i}, {k})"
                ))

# Add stations (blue dots)
fig.add_trace(go.Scatter(
    x=[x for x, _, _ in coords_plot],
    y=[y for _, y, _ in coords_plot],
    mode="markers",
    marker=dict(color="blue", size=10),
    text=[f"Station: {s}<br>X: {x:.2f}<br>Y: {y:.2f}" for x, y, s in coords_plot],
    hoverinfo="text",
    showlegend=False
))

# Update layout
fig.update_layout(
    title="Stations and arcs",
    width=1000, height=1000,
    xaxis=dict(
        range=[-1.1, 1.1],
        showgrid=False,
        zeroline=False,
    ),
    yaxis=dict(
        range=[-1.1, 1.1],
        showgrid=False,
        zeroline=False,
        scaleanchor="x",
        scaleratio=1
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.01
    ),
    hovermode="closest"
)

fig.show()

In [11]:
performance

{'runtime': 0.07273697853088379,
 'nodes': 21.0,
 'iterations': 177.0,
 'gap': 0.0,
 'obj': 40087.146130013745}

In [12]:
print(rail)

Railway scheduling problem

Parameters:
N:  10 stations
T:  10 periods
J:  10 jobs
P:  2000 passengers
K:  3 alternative routes
Aj: 10 jobs with arcs
Ja: 9 arcs with jobs
C:  0 arcs unavailable simultaneously

Optimization model:
Variables:   4600
Constraints: 9805
Objective:   -3513895.9717796026 + 1746.0 v[1,2,1] + 772.0 v[1,2,2] + 1800.0 v[1,2,3] + 223.0 v[1,2,4] + 167.0 v[1,2,5] + 1855.0 v[1,2,6] + 1169.0 v[1,2,7] + 1993.0 v[1,2,8] + 869.0 v[1,2,9] + 1468.0 v[1,2,10] + 76.0 v[1,3,1] + 1974.0 v[1,3,2] + 561.0 v[1,3,3] + 301.0 v[1,3,4] + 442.0 v[1,3,5] + 894.0 v[1,3,6] + 1510.0 v[1,3,7] + 1160.0 v[1,3,8] + 851.0 v[1,3,9] + 1030.0 v[1,3,10] + 1478.0 v[1,4,1] + 581.0 v[1,4,2] + 1750.0 v[1,4,3] + 1823.0 v[1,4,4] + 367.0 v[1,4,5] + 1950.0 v[1,4,6] + 988.0 v[1,4,7] + 1695.0 v[1,4,8] + 1028.0 v[1,4,9] + 808.0 v[1,4,10] + 465.0 v[1,5,1] + 1128.0 v[1,5,2] + 1081.0 v[1,5,3] + 908.0 v[1,5,4] + 1816.0 v[1,5,5] + 790.0 v[1,5,6] + 333.0 v[1,5,7] + 482.0 v[1,5,8] + 892.0 v[1,5,9] + 1081.0 v[1,5,10

In [13]:
print(rail2)

Railway scheduling problem

Parameters:
N:  10 stations
T:  10 periods
J:  10 jobs
P:  2000 passengers
K:  3 alternative routes
Aj: 10 jobs with arcs
Ja: 9 arcs with jobs
C:  0 arcs unavailable simultaneously

Optimization model:
Variables:   0
Constraints: 0
Objective:   0.0
Status:      LOADED



In [14]:
# Measure runtime
runtime = rail.model.Runtime
print(f"Optimization took {runtime:.4f} seconds")

Optimization took 0.0727 seconds


In [15]:
print(type(rail.S))
print(rail.S)

print()

print(type(rail.Aj))
print(rail.Aj)

print()

print(type(rail.A))
print(rail.A)

<class 'dict'>
{1: 9, 2: 8, 3: 1, 4: 1, 5: 1, 6: 1, 7: 6, 8: 4, 9: 1, 10: 2}

<class 'dict'>
{1: [(9, 10)], 2: [(7, 10)], 3: [(2, 6)], 4: [(1, 6)], 5: [(3, 10)], 6: [(8, 9)], 7: [(6, 10)], 8: [(7, 10)], 9: [(2, 9)], 10: [(4, 8)]}

<class 'list'>
[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (6, 8), (6, 9), (6, 10), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10)]


In [16]:
S = rail.S.copy()
Aj_original = rail.Aj.copy()  # Keep original reference
Aj = rail.Aj.copy() 
A = rail.A.copy()
Tend = rail.Tend

# Create dictionary of named arcs
An = {}
for i, a in enumerate(A):
    An[f"a{i}"] = a

# Use a regular set instead of OrderedDict
jobs_arc_set = set()

# Convert arcs in Aj dictionary to their names
for job, job_arc_list in Aj_original.items():
    for i, arc in enumerate(job_arc_list):
        # Find the name of the arc in An
        arc_name = [name for name, value in An.items() if value == arc]
        if arc_name:
            # Replace the arc with its name
            Aj[job][i] = arc_name[0]
            
            # Add the arc to the set
            jobs_arc_set.add(arc_name[0])

# Sort the set when you need ordered output
# This sorts by the numerical portion (e.g., "a12" -> 12)
sorted_arc_names = sorted(jobs_arc_set, key=lambda x: int(x[1:]))

# Convert arcs in Aj to their position in sorted_arc_names
for job, job_arc_list in Aj.items():
    for i, arc_name in enumerate(job_arc_list):
        # Find the index of the arc name in sorted_arc_names
        index = sorted_arc_names.index(arc_name)
        # Replace the arc name with its index
        Aj[job][i] = index
        
        
    
print('sorted_arc_names')
print(sorted_arc_names)

print('Aj')
print(Aj)

print('S')
print(S)

sorted_arc_names
['a4', 'a12', 'a15', 'a23', 'a27', 'a38', 'a41', 'a42', 'a44']
Aj
{1: [8], 2: [6], 3: [1], 4: [0], 5: [3], 6: [7], 7: [5], 8: [6], 9: [2], 10: [4]}
S
{1: 9, 2: 8, 3: 1, 4: 1, 5: 1, 6: 1, 7: 6, 8: 4, 9: 1, 10: 2}


In [17]:
j0 = list(S.keys())[0]
Aj[j0]

[8]

In [18]:
import numpy as np
import plotly.graph_objects as go

fig = go.Figure()

# Collect arcs data
num_arcs = len(sorted_arc_names)
label_positions = [i + 0.5 for i in range(len(sorted_arc_names))]

# Horizontal white lines at every integer y value
for i in range(num_arcs + 1):
    fig.add_shape(type="line",
        x0=0, y0=i, x1=Tend, y1=i,
        line=dict(
            color='white',
            width=1,
        ),
    )


# Get color palette for jobs
colors = [
    "LightSkyBlue", "LightCoral", "LightGreen", "LightPink",
    "LightSalmon", "LightSteelBlue", "LightYellow", "LightGray",
    "LightGoldenRodYellow", "LightCyan"
]

# Add shapes
# j0 = list(S.keys())[0]
# fig.add_shape(type="rect",
#     x0=S[j0],
#     y0=Aj[j0][0],
#     x1=S[j0] + 1,
#     y1=Aj[j0][0] + 1,
#     line=dict(
#         color="RoyalBlue",
#         width=1,
#     ),
#     fillcolor="LightSkyBlue",
# )
# fig.update_shapes(dict(xref='x', yref='y'))

jobs = list(S.keys())
for j in jobs:
    for i in range(len(Aj[j])):
        fig.add_shape(type="rect",
            x0=S[j],
            y0=Aj[j][i],
            x1=S[j] + 1,
            y1=Aj[j][i] + 1,
            line=dict(
                color=colors[jobs.index(j) % len(colors)],
                width=1,
            ),
            fillcolor=colors[jobs.index(j) % len(colors)],
            # hovertext=f"Job {j} - Arc {sorted_arc_names[Aj[j][i]]}",
        )
        fig.update_shapes(dict(xref='x', yref='y'))


# Layout
fig.update_layout(
    width=800,
    height=600,
    xaxis=dict(
        title="Time",
        range=[0, Tend],
        tickvals=list(range(0, Tend + 1)),
    ),
    yaxis=dict(
        type="category",
        title="Arcs",
        range=[0, num_arcs],
        tickvals=label_positions,
        ticktext=sorted_arc_names,
        showgrid=False,
    ),
)


fig.show()

# # Save as HTML
# fig.write_html("railway_schedule.html")

In [20]:
import numpy as np
import plotly.graph_objects as go

fig = go.Figure()

# Collect arcs data
num_arcs = len(sorted_arc_names)
label_positions = [i + 0.5 for i in range(len(sorted_arc_names))]

# Horizontal white lines at every integer y value
for i in range(num_arcs + 1):
    fig.add_shape(type="line",
        x0=0, y0=i, x1=Tend, y1=i,
        line=dict(
            color='white',
            width=1,
        ),
    )

# Get color palette for jobs
colors = [
    "LightSkyBlue", "LightCoral", "LightGreen", "LightPink",
    "LightSalmon", "LightSteelBlue", "LightYellow", "LightGray",
    "LightGoldenRodYellow", "LightCyan"
]

# Add shapes and hover points
jobs = list(S.keys())
hover_x = []
hover_y = []
hover_text = []
hover_colors = []

for j in jobs:
    for i in range(len(Aj[j])):
        # Add rectangle shape
        fig.add_shape(type="rect",
            x0=S[j],
            y0=Aj[j][i],
            x1=S[j] + 1,
            y1=Aj[j][i] + 1,
            line=dict(
                color=colors[jobs.index(j) % len(colors)],
                width=1,
            ),
            fillcolor=colors[jobs.index(j) % len(colors)],
        )
        
        # Collect hover data for this rectangle
        hover_x.append(S[j] + 0.5)  # Center of rectangle
        hover_y.append(Aj[j][i] + 0.5)  # Center of rectangle
        hover_text.append(f"Job {j} - Arc {sorted_arc_names[Aj[j][i]]}")
        hover_colors.append(colors[jobs.index(j) % len(colors)])

# Add invisible scatter trace with hover information
fig.add_trace(go.Scatter(
    x=hover_x,
    y=hover_y,
    mode="markers",
    marker=dict(
        size=15,
        color=hover_colors,
        opacity=0,  # Make markers invisible
    ),
    text=hover_text,
    hoverinfo="text",
))

# Layout
fig.update_layout(
    # width=800,
    # height=600,
    autosize=True,   # let the container size drive the plot
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis=dict(
        title="Time",
        range=[0, Tend],
        tickvals=list(range(0, Tend + 1)),
    ),
    yaxis=dict(
        type="category",
        title="Arcs",
        range=[0, num_arcs],
        tickvals=label_positions,
        ticktext=sorted_arc_names,
        showgrid=False,
    ),
)
plot(
  fig,
  filename='railway_schedule.html',
  auto_open=False,
  include_plotlyjs=True,
  config={'responsive': True}
)


# fig.show()

# # Save as HTML
# fig.write_html("railway_schedule.html")

'railway_schedule.html'